In [1]:
import os

In [2]:
%pwd

'e:\\Python Projects\\Deep_learning\\mlops_test1\\research'

In [3]:
os.chdir("../")

update Config

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen= True)
class TrainingConfig:
    root_directory: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

@dataclass(frozen= True)
class PrepareCallbacksConfig:
    root_directory: Path
    tensorboard_root_log_directory: Path
    checkpoint_model_filepath: Path


update the configuration Manager

In [5]:
from src.chicken_disease_classification.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH
from src.chicken_disease_classification.utils.common import read_yaml,create_directories
import tensorflow as tf

In [6]:
class ConfigurationManager:
    def __init__(self,
                config_file_path,
                params_file_path):
        self.config= read_yaml(config_file_path)
        self.params= read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])

    def get_prepare_callbacks_config(self)-> PrepareCallbacksConfig :
        config= self.config.prepare_callbacks
        model_chkpt_dir= os.path.dirname(config.checkpoint_model_filepath)
        create_directories([Path(model_chkpt_dir),Path(config.tensorboard_root_log_dir)])

        prepare_callbacks_config = PrepareCallbacksConfig(
            root_directory=Path(config.root_directory),
            tensorboard_root_log_directory= Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath= Path(config.checkpoint_model_filepath)
        )
        return prepare_callbacks_config
    
    def training_config(self) -> TrainingConfig:
        training= self.config.training
        prepare_base_model= self.config.prepare_base_model
        params= self.params
        training_data= os.path.join(self.config.data_ingestion.unzip_dir,"Chicken-fecal-images")
        
        create_directories([Path(training.root_directory)])
        
        training_config= TrainingConfig(
            root_directory= Path(training.root_directory),
            trained_model_path= Path(training.model_file_path),
            updated_base_model_path= Path(prepare_base_model.updated_base_model_path),
            training_data= Path(training_data),
            params_epochs= params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config


Create component

In [7]:
import time

In [8]:
class PrepareCallbacks:
    def __init__(self, config:PrepareCallbacksConfig):
        self.config= config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_directory,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


        
    

In [9]:
import tensorflow as tf
import numpy as np
from zipfile import ZipFile
import urllib.request as request
import os
import time

In [10]:
class Training:
    def __init__(self,config: TrainingConfig):
        self.config= config

    def get_base_model(self):
        self.model= tf.keras.models.load_model(
            self.config.updated_base_model_path
            )
        
    
    def train_valid_generator(self):
        datagenerator_kwargs={
            'rescale': 1/255,
            'validation_split': 0.20
 
        }

        dataflow_kwargs= dict(
            target_size= self.config.params_image_size[:-1],
            batch_size= self.config.params_batch_size,
            interpolation='bilinear'
        )

        valid_datagenerator= tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

        self.valid_generator= valid_datagenerator.flow_from_directory(
            directory= self.config.training_data,
            subset="validation",
            shuffle= False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train__datagenerator= tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range= 40,
                horizontal_flip= True,
                width_shift_range= 0.2,
                height_shift_range= 0.2,
                shear_range= 0.2,
                zoom_range= 0.2,
                **datagenerator_kwargs
            )
        else:
            train__datagenerator= valid_datagenerator

        self.train_generator= train__datagenerator.flow_from_directory(
            directory= self.config.training_data,
            subset= 'training',
            shuffle= True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self, callback_list: list):
        self.steps_per_epoch= self.train_generator.samples//self.train_generator.batch_size
        self.validation_steps= self.valid_generator.samples//self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs= self.config.params_epochs,
            steps_per_epoch= self.steps_per_epoch,
            validation_steps= self.validation_steps,
            validation_data= self.valid_generator,
            callbacks= callback_list

        )

        self.save_model(
            path= self.config.trained_model_path,
            model= self.model
        )

In [11]:
# create the pipeline
from src.chicken_disease_classification.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH

In [12]:
try:
    config_manager= ConfigurationManager(config_file_path= CONFIG_FILE_PATH,params_file_path=PARAMS_FILE_PATH)
    prepare_callbacks_config= config_manager.get_prepare_callbacks_config()
    prepare_callback_component= PrepareCallbacks(config= prepare_callbacks_config)
    callback_list= prepare_callback_component.get_tb_ckpt_callbacks()

    training_config= config_manager.training_config()
    training_component= Training(config= training_config)
    training_component.get_base_model()
    training_component.train_valid_generator()
    training_component.train(
        callback_list= callback_list
    )

except Exception as e:
    raise e

2025-05-18 13:54:55,952 - INFO - yaml file: e:\Python Projects\Deep_learning\mlops_test1\config\config.yaml loaded successfully
2025-05-18 13:54:55,952 - INFO - yaml file: e:\Python Projects\Deep_learning\mlops_test1\params.yaml loaded successfully
2025-05-18 13:54:55,959 - INFO - created directory at: artifacts
2025-05-18 13:54:55,959 - INFO - created directory at: artifacts\prepare_callbacks\checkpoint_dir
2025-05-18 13:54:55,962 - INFO - created directory at: artifacts\prepare_callbacks\tensorboard_log_dir
2025-05-18 13:54:55,966 - INFO - created directory at: artifacts\training
2025-05-18 13:54:56,200 - WARNING - No training configuration found in the save file, so the model was *not* compiled. Compile it manually.
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.


ValueError: You must call `compile()` before using the model.